In [1]:
from platform import python_version

print(python_version())

3.7.12


In [2]:
import findspark
findspark.init()
findspark.find()

'C:\\spark'

In [3]:
import pyspark 

In [9]:
import os
import pandas as pd
import numpy as np

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext

from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col

from pyspark.ml.regression import LinearRegression
from pyspark.mllib.evaluation import RegressionMetrics

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.evaluation import RegressionEvaluator

In [5]:
import pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["mydatabase"]
mycol = mydb["customers"]



In [81]:
mydict = { "text":"se référant à l'ensemble des altérations de","method":200 }

x = mycol.insert_one(mydict)

In [82]:
import json
x = mycol.find()
l=[]

for data in x:
    new_list = list(data.values())
    new_list.pop(0)
    
    l.append(new_list)
    
df1=pd.DataFrame(l ,columns=["text","method"])
df1

,text,method
0,link line,404
1,sport jeux,200
2,clée usb,300
3,La maladie est une altération,200
4,se référant à l'ensemble des altérations de,200


In [84]:
from pyspark.sql import SparkSession
spark = SparkSession(sc).builder.appName("TP_ML_Email_spam").getOrCreate()

In [85]:
spark = SparkSession.builder.master("local[2]").appName("Linear-Regression-California-Housing").getOrCreate()

In [86]:
# define the schema, corresponding to a line in the csv data file.
schema = StructType([
    StructField("text", StringType(), nullable=True),
    StructField("class", IntegerType(), nullable=True)]
)

In [87]:
df = spark.createDataFrame (df1, schema=schema)

In [88]:
df.show(5)

+--------------------+-----+
|                text|class|
+--------------------+-----+
|           link line|  404|
|          sport jeux|  200|
|            clée usb|  300|
|La maladie est un...|  200|
|se référant à l'e...|  200|
+--------------------+-----+



In [90]:
df = df.filter((df['class']=="404")|(df['class']=="200"))

In [91]:
df.show(5)

+--------------------+-----+
|                text|class|
+--------------------+-----+
|           link line|  404|
|          sport jeux|  200|
|La maladie est un...|  200|
|se référant à l'e...|  200|
+--------------------+-----+



In [92]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer

In [93]:
tokenizer = Tokenizer(inputCol="text", outputCol="words")

In [94]:
regexTokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="\\W")

In [95]:
regexTokenized = regexTokenizer.transform(df)

In [96]:
tokenized = tokenizer.transform(df)

In [97]:
tokenized.show(5)

+--------------------+-----+--------------------+
|                text|class|               words|
+--------------------+-----+--------------------+
|           link line|  404|        [link, line]|
|          sport jeux|  200|       [sport, jeux]|
|La maladie est un...|  200|[la, maladie, est...|
|se référant à l'e...|  200|[se, référant, à,...|
+--------------------+-----+--------------------+



In [98]:
from pyspark.ml.feature import StopWordsRemover

In [99]:
remover = StopWordsRemover(inputCol="words", outputCol="filtered")

In [100]:
data = remover.transform(regexTokenized)

In [101]:
data.show(5)

+--------------------+-----+--------------------+--------------------+
|                text|class|               words|            filtered|
+--------------------+-----+--------------------+--------------------+
|           link line|  404|        [link, line]|        [link, line]|
|          sport jeux|  200|       [sport, jeux]|       [sport, jeux]|
|La maladie est un...|  200|[la, maladie, est...|[la, maladie, est...|
|se référant à l'e...|  200|[se, r, f, rant, ...|[se, r, f, rant, ...|
+--------------------+-----+--------------------+--------------------+



In [102]:
data.select('words').first()

Row(words=['link', 'line'])

In [103]:
data.select('filtered').first()

Row(filtered=['link', 'line'])

In [104]:
def mots(txt):
    return [w for w in txt if ((len(w) > 2) & (w != 'subject'))  ]

In [105]:
str = ["subject","sam","de","fat","*","--"]
print(mots(str))

['sam', 'fat']


In [106]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType
mots = udf(mots,ArrayType(StringType()))

In [107]:
toNum = lambda str:int(str)

In [108]:
toNum('3')+2

5

In [109]:
toNum = udf(toNum,'int')

In [110]:
data2 = data.withColumn('motsPur', mots(data["filtered"]))

In [111]:
data2.show(2)

+----------+-----+-------------+-------------+-------------+
|      text|class|        words|     filtered|      motsPur|
+----------+-----+-------------+-------------+-------------+
| link line|  404| [link, line]| [link, line]| [link, line]|
|sport jeux|  200|[sport, jeux]|[sport, jeux]|[sport, jeux]|
+----------+-----+-------------+-------------+-------------+
only showing top 2 rows



In [112]:
data2 = data2.withColumn('label', toNum(data["class"]))

In [113]:
data2.show(2)

+----------+-----+-------------+-------------+-------------+-----+
|      text|class|        words|     filtered|      motsPur|label|
+----------+-----+-------------+-------------+-------------+-----+
| link line|  404| [link, line]| [link, line]| [link, line]|  404|
|sport jeux|  200|[sport, jeux]|[sport, jeux]|[sport, jeux]|  200|
+----------+-----+-------------+-------------+-------------+-----+
only showing top 2 rows



In [114]:
data2.select('motsPur').first()

Row(motsPur=['link', 'line'])

In [115]:
data2.select('label').show(5)

+-----+
|label|
+-----+
|  404|
|  200|
|  200|
|  200|
+-----+



In [116]:
from pyspark.ml.feature import CountVectorizer

In [117]:
# fit a CountVectorizerModel from the corpus.
cv = CountVectorizer(inputCol="motsPur", outputCol="features")
data3 = cv.fit(data2).transform(data2)

In [121]:
train, test = data3.select('features','label').randomSplit([0.5,0.5],seed=100)

In [122]:
train.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(13,[4,6],[1.0,1.0])|  200|
|(13,[0,5,8,11,12]...|  200|
+--------------------+-----+



In [123]:
test.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(13,[2,3],[1.0,1.0])|  404|
|(13,[0,1,7,9,10],...|  200|
+--------------------+-----+



In [124]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [125]:
# create the trainer and set its parameters
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")
# train the model
model = nb.fit(train)

In [126]:
# select example rows to display.
predictions = model.transform(test)
#predictions.show()

In [127]:
# select example rows to display.
test2 = test.select('features')
predictions2 = model.transform(test2)
#predictions2.show()

In [128]:
# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")

In [129]:
accuracy = evaluator.evaluate(predictions)

In [130]:
accuracy

0.0

In [131]:
evaluator.evaluate(predictions)

0.0